# Movie Sentiment Analysis with Keras

In [ ]:
!pip install nltk
!pip install --upgrade gensim

import numpy as np
import os
import os.path

from pdb import set_trace
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
import nltk


import glob
from gensim.models import Word2Vec

import time

     |████████████████████████████████| 23.9 MB 92 kB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [ ]:
# MacOSX: See https://www.mkyong.com/mac/wget-on-mac-os-x/ for wget
if not os.path.isdir('./aclImdb'):
    if not os.path.isfile('./aclImdb_v1.tar.gz'):
      !wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz 

    if not os.path.isdir('./aclImdb'):  
      !tar -xf aclImdb_v1.tar.gz 

--2021-08-27 20:02:32--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  19.7MB/s    in 5.5s    

2021-08-27 20:02:37 (14.6 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [ ]:
time_beginning_of_notebook = time.time()
# SAMPLE_SIZE=12500
SAMPLE_SIZE=50000 #4000

positive_sample_file_list_test = glob.glob(os.path.join('./aclImdb/test/pos', "*.txt"))
positive_sample_file_list = glob.glob(os.path.join('./aclImdb/train/pos', "*.txt"))
#positive_sample_file_list = positive_sample_file_list[:SAMPLE_SIZE]

negative_sample_file_list_test = glob.glob(os.path.join('./aclImdb/test/neg', "*.txt"))
negative_sample_file_list = glob.glob(os.path.join('./aclImdb/train/neg', "*.txt"))
#negative_sample_file_list = negative_sample_file_list[:SAMPLE_SIZE]

import re

# load doc into memory
# regex to clean markup elements 
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r', encoding='utf8')
    # read all text
    text = re.sub('<[^>]*>', ' ', file.read())
    #text = file.read()
    # close the file
    file.close()
    return text


In [ ]:
import pandas as pd
from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

df_positives = pd.DataFrame({'reviews':[load_doc(x) for x in positive_sample_file_list], 'sentiment': np.ones(len(positive_sample_file_list))}) #(SAMPLE_SIZE)
df_negatives = pd.DataFrame({'reviews':[load_doc(x) for x in negative_sample_file_list], 'sentiment': np.zeros(len(negative_sample_file_list))})

print("Positive review(s):", df_positives['reviews'][1][:100])
print("Negative review(s):", df_negatives['reviews'][1][:100])

df = pd.concat([df_positives, df_negatives], ignore_index=True)

df = shuffle(df)

##

df_positives_test = pd.DataFrame({'reviews':[load_doc(x) for x in positive_sample_file_list_test], 'sentiment': np.ones(len(positive_sample_file_list))})
df_negatives_test = pd.DataFrame({'reviews':[load_doc(x) for x in negative_sample_file_list_test], 'sentiment': np.zeros(len(negative_sample_file_list))})

print("Positive review(s)_test:", df_positives_test['reviews'][1][:100])
print("Negative review(s)_test:", df_negatives_test['reviews'][1][:100])

df_test = pd.concat([df_positives_test, df_negatives_test], ignore_index=True)

df_test = shuffle(df_test)


#X_train, X_test, y_train, y_test = train_test_split(df['reviews'], df['sentiment'], test_size=0.25)
X_train, y_train = df['reviews'], df['sentiment']
X_test, y_test = df_test['reviews'], df_test['sentiment']




Positive review(s): This is my first comment on IMDb website, and the reason I'm writing it is that we're talking about 
Negative review(s): The only thing that kept me from vomiting after seeing this movie was the fact that these are just a
Positive review(s)_test: if you watch this at home on DVD or Bluray! be sure you have great sound system. the musical score t
Negative review(s)_test: What can be said about a movie that makes two hours seem like three weeks? The hero starts out in ni


In [ ]:
X_train

21979    The Sentinel represents everything about the s...
4078     "This Is Not A Love Song" is a brilliant examp...
6903     This is probably the best documentary I have s...
22835    I caught this a few months ago on Family Chann...
4629     I first saw "Breaking Glass" when it was relea...
                               ...                        
20942    Be warned: Neither Zeta-Jones nor McGregor pla...
3718     I caught this movie on the Sundance Channel on...
12049    I enjoyed the innocence of this film and how t...
4976     This is an intimate movie of a sincere girl in...
20566    This movie had mediocrity, laziness, and thoug...
Name: reviews, Length: 25000, dtype: object

In [ ]:
X_test


1249     hello, looking for a movie for the family to w...
7591     ...the opportunity it gave me to look at Irela...
17411    Overall, this is a pretty bad film. But for $5...
23886    I was in Blockbuster and I saw a film called "...
18150    Ahh, nuthin' like cheesy, explopitative, semi-...
                               ...                        
8169     "Entrails of a Beauty" features a gang of Yaku...
5760     Talkshow with Spike Feresten is one of those s...
2725     [Minor spoilers follow]  Steve Allen opined th...
15257    This music is totally out of touch with the fi...
4947     I watched Pola X because Scott Walker composed...
Name: reviews, Length: 25000, dtype: object

In [ ]:
#ML STUDY GROUP
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

class PreProcessor:
    def __init__(self,REVIEWS,REVIEWS_VAL,LABELS,LABELS_VAL,WE_FILE):
        self.reviews = REVIEWS
        self.reviews_val = REVIEWS_VAL
        self.labels = LABELS
        self.labels_val = LABELS_VAL
        self.we_file = WE_FILE

    def tokenize(self):
#         set_trace()
        print(self.reviews[0])

        tokenizer = Tokenizer()
        tokenizer.fit_on_texts(self.reviews)

        self.sequences = tokenizer.texts_to_sequences(self.reviews)
        self.sequences_val = tokenizer.texts_to_sequences(self.reviews_val)

        self.word_index = tokenizer.word_index
        print("Found %s unique tokens" %(len(self.word_index)))

    def make_data(self):
        self.MAX_SEQUENCE_LENGTH = max([len(self.sequences[i]) for i in range(len(self.sequences))])
        print("self.MAX_SEQUENCE_LENGTH: {}".format(self.MAX_SEQUENCE_LENGTH))

        review = pad_sequences(self.sequences,maxlen=self.MAX_SEQUENCE_LENGTH)
        review_val = pad_sequences(self.sequences_val,maxlen=self.MAX_SEQUENCE_LENGTH)
        
        labels = to_categorical(self.labels)
        labels_val = to_categorical(self.labels_val)

        print("Shape of data tensor: " +str(review.shape))
        print("Shape of label tensor: " +str(labels.shape))

        return review, review_val, labels, labels_val
        
    def get_word_embedding_matrix(self,EMBEDDING_DIM=100):
        embeddings_index = {}

        if self.we_file == "rand":
            return None

        f = open(self.we_file)

        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
        f.close()

        print('Found %s word vectors.' % len(embeddings_index))

        self.embedding_matrix = np.zeros((len(self.word_index)+1, EMBEDDING_DIM))

        for word, i in self.word_index.items():
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                # words not found in embedding index will be all-zeros.
                self.embedding_matrix[i] = embedding_vector

        return self.embedding_matrix


In [ ]:
# MacOSX: See https://www.mkyong.com/mac/wget-on-mac-os-x/ for wget
if not os.path.isfile('./glove.6B.300d.txt'):
    if not os.path.isfile('./glove.6B.zip'):
      !wget http://nlp.stanford.edu/data/glove.6B.zip 

    if not os.path.isfile('./glove.6B.300d.txt'):  
      !unzip glove.6B.zip 
      

--2021-08-27 20:14:15--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-08-27 20:14:16--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-08-27 20:14:16--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
import argparse
import numpy as np

from keras.layers import Embedding, Dense, Input, BatchNormalization, Activation, Dropout
from keras.models import Sequential
from keras.optimizers import adagrad_v2, adam_v2
from keras import backend as K

from pdb import set_trace

embedding_dim = 300
num_hidden_layers = 3
num_hidden_units = 300
num_epochs = 100
batch_size = 512
dropout_rate = 0.2
word_dropout_rate = 0.3
activation = 'relu'

args = {}
args['We']='./glove.6B.300d.txt'
args['Wels']='' ### rand or ''
args['model']='dan'  ### nbow OR dan
args['wd']='y'

# reviews=X_train.values
# reviews_val=X_test.values
# labels=y_train.values
# labels_val=y_test.values

In [ ]:
X_train

21979    The Sentinel represents everything about the s...
4078     "This Is Not A Love Song" is a brilliant examp...
6903     This is probably the best documentary I have s...
22835    I caught this a few months ago on Family Chann...
4629     I first saw "Breaking Glass" when it was relea...
                               ...                        
20942    Be warned: Neither Zeta-Jones nor McGregor pla...
3718     I caught this movie on the Sundance Channel on...
12049    I enjoyed the innocence of this film and how t...
4976     This is an intimate movie of a sincere girl in...
20566    This movie had mediocrity, laziness, and thoug...
Name: reviews, Length: 25000, dtype: object

In [ ]:
pp = PreProcessor(X_train,X_test,y_train,y_test,args['We'])
pp.tokenize()

encoded_X_train,encoded_X_test,y_train,y_test = pp.make_data()

embedding_matrix = pp.get_word_embedding_matrix(embedding_dim)


The most hillarious and funny Brooks movie I ever seen. I can watch and re-watch the tape 100 times. I laugh my a** off and I cry on some moments. It is really good and funny movie, and if you like Brooks - this is a must! In short - Brooks (billionare) gets to the streets as homeless for 30 days in order to win the entire poor district from his competitor. The reality bites, but in the end - it is about warm relations between humans... Hightly recommend!
Found 88576 unique tokens
self.MAX_SEQUENCE_LENGTH: 2473
Shape of data tensor: (25000, 2473)
Shape of label tensor: (25000, 2)
Found 400000 word vectors.


In [ ]:
embedding_matrix.shape
# pp.MAX_SEQUENCE_LENGTH
len(pp.word_index)+1
embedding_dim

300

https://machinelearningmastery.com/develop-n-gram-multichannel-convolutional-neural-network-sentiment-analysis/

In [ ]:
from keras.layers import Concatenate
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate
# create the model

channels = []
inputs = []
encoded_X_trains= []
encoded_X_tests = []
for filter_len in [3,4,5]:
    inputs1 = Input(shape=(pp.MAX_SEQUENCE_LENGTH,))
    inputs.append(inputs1)
    embedding1 = Embedding(len(pp.word_index)+1,embedding_dim,weights=[embedding_matrix],\
                           input_length=pp.MAX_SEQUENCE_LENGTH,trainable=True)(inputs1)
    conv1 = Conv1D(filters=128, kernel_size=filter_len, padding='same', activation='relu')(embedding1)
    drop1 = Dropout(0.5)(conv1)
    pool1 = MaxPooling1D(pool_size=2)(drop1)
    flat1 = Flatten()(pool1)
    channels.append(flat1)
    encoded_X_trains.append(encoded_X_train)
    encoded_X_tests.append(encoded_X_test)
    
# merge
merged = concatenate(channels)
# interpretation
outputs = Dense(2, activation='softmax')(merged)
model = Model(inputs=inputs, outputs=outputs)
# compile
    
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy','categorical_accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 2473)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 2473)]       0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 2473)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 2473, 300)    26573100    input_1[0][0]                    
______________________________________________________________________________________________

In [ ]:
%pdb off
batch_size = 64
# num_epochs = 3
num_epochs = 4

history = model.fit(encoded_X_trains,y_train,batch_size=batch_size,epochs=num_epochs,\
          validation_data=(encoded_X_tests,y_test))


Automatic pdb calling has been turned OFF
Epoch 1/4
391/391 [==============================] - 227s 502ms/step - loss: 0.4354 - accuracy: 0.8027 - categorical_accuracy: 0.8027 - val_loss: 0.2745 - val_accuracy: 0.8910 - val_categorical_accuracy: 0.8910
Epoch 2/4
391/391 [==============================] - 195s 499ms/step - loss: 0.1612 - accuracy: 0.9402 - categorical_accuracy: 0.9402 - val_loss: 0.2862 - val_accuracy: 0.8890 - val_categorical_accuracy: 0.8890
Epoch 3/4
391/391 [==============================] - 195s 500ms/step - loss: 0.0640 - accuracy: 0.9787 - categorical_accuracy: 0.9787 - val_loss: 0.3569 - val_accuracy: 0.8796 - val_categorical_accuracy: 0.8796
Epoch 4/4
391/391 [==============================] - 195s 499ms/step - loss: 0.0348 - accuracy: 0.9889 - categorical_accuracy: 0.9889 - val_loss: 0.3929 - val_accuracy: 0.8824 - val_categorical_accuracy: 0.8824


y_train

In [ ]:
df = pd.DataFrame(history.history)
df=df[df['val_accuracy']==df.val_accuracy.max()]
df.reset_index(inplace=True)
df["title"]=["Keras CNN with pretrained embedding"]
df["sample_size"]=[SAMPLE_SIZE]
df["nb_epochs"]=[df.iloc[0]["index"]+1]
df.drop(labels="index",axis=1,inplace=True)
print(df)
# df.to_csv(path_or_buf=df.iloc[0].title+".csv")

       loss  accuracy  ...  sample_size  nb_epochs
0  0.435447   0.80272  ...        50000          1

[1 rows x 9 columns]


In [ ]:
df

,loss,accuracy,categorical_accuracy,val_loss,val_accuracy,val_categorical_accuracy,title,sample_size,nb_epochs
0,0.435447,0.80272,0.80272,0.27448,0.891,0.891,Keras CNN with pretrained embedding,50000,1
